# Analysis

## How to analyze facial expression data with Feat.

*Written by Jin Hyun Cheong*

Here we will be using a sample dataset by David Watson on ["A Data-Driven Characterisation Of Natural Facial Expressions When Giving Good And Bad News"](https://journals.plos.org/ploscompbiol/article/peerReview?id=10.1371/journal.pcbi.1008335) by Watson & Johnston 2020. The full dataset is available on [OSF](https://osf.io/6tbwj/).

First, we download the necessary files & videos. 

In [5]:
import subprocess
files_to_download = ["4c5mb", "n6rt3", "3gh8v", "twqxs", "nc7d9", "nrwcm", "2rk9c", "mxkzq", "c2na7", "wj7zy", "mxywn", 
                     "6bn3g", "jkwsp", "54gtv", "c3hpm", "utdqj", "hpw4a", "94swe", "qte5y", "aykvu", "3d5ry"]

for fid in files_to_download:
    subprocess.run(f"wget --content-disposition https://osf.io/{fid}/download".split())

Check that videos have been downloaded and the attributes file, `clip_attrs.csv) explaining 

In [1]:
import os, glob
import numpy as np
import pandas as pd

clip_attrs = pd.read_csv("clip_attrs.csv")
videos = np.sort(glob.glob("*.mp4"))
print(videos)

['001.mp4' '002.mp4' '003.mp4' '004.mp4' '005.mp4' '006.mp4' '007.mp4'
 '008.mp4' '009.mp4' '010.mp4' '011.mp4' '012.mp4' '013.mp4' '014.mp4'
 '015.mp4' '016.mp4' '017.mp4' '018.mp4' '019.mp4' '020.mp4']


Process each video using our detector. 

In [ ]:
from feat import Detector
detector = Detector(au_model = "jaanet", emotion_model = "resmasknet")
for video in videos: 
    detector.detect_video(video, outputFname = video.replace(".mp4", ".csv"))

In [14]:
from feat.utils import read_feat
import pandas as pd

for ix ,video in enumerate(videos):
    outputF = video.replace(".mp4", ".csv")
    if ix == 0: 
        fex = read_feat(outputF)
    else:
        fex = pd.concat([out, read_feat(outputF)])
fex = fex.dropna()

In [28]:
# Load in conditions
clip_attrs = pd.read_csv("clip_attrs.csv")
clip_attrs["input"] = clip_attrs.clipN.apply(lambda x: str(x).zfill(3)+".mp4")
input_class_map = dict(zip(clip_attrs.input, clip_attrs['class']))


## Extract features

In [49]:
fex.sessions = fex.input()
average_au_intensity_per_video = fex.extract_mean()
display(average_au_intensity_per_video.head())

,mean_AU01,mean_AU02,mean_AU04,mean_AU06,mean_AU07,mean_AU10,mean_AU12,mean_AU14,mean_AU15,mean_AU17,...,mean_y_61,mean_y_62,mean_y_63,mean_y_64,mean_y_65,mean_y_66,mean_y_67,mean_y_7,mean_y_8,mean_y_9
001.mp4,0.140665,0.855856,0.070829,0.851965,0.748190,0.842554,0.992311,0.524729,0.261690,0.000499,...,694.305659,696.236139,692.150659,682.592831,696.408174,700.882806,699.187324,814.856982,819.742603,808.427911
002.mp4,0.068047,0.757220,0.065335,0.756616,0.593310,0.892189,0.996428,0.528385,0.474023,0.000575,...,686.464658,688.592334,685.344299,677.572549,688.540245,692.234408,690.349490,806.882534,812.669819,802.669351
003.mp4,0.097756,0.829843,0.056197,0.777233,0.786006,0.870444,0.995471,0.662425,0.470681,0.000654,...,678.527397,681.056783,677.828855,672.959971,688.190226,691.792301,689.539840,803.369594,810.371285,800.508759
004.mp4,0.245191,0.899495,0.120592,0.870192,0.821127,0.832403,0.984547,0.558193,0.228535,0.000828,...,682.179613,684.565805,680.513723,674.068777,694.367983,698.833575,696.740101,811.564575,817.511093,806.248735
005.mp4,0.062102,0.664194,0.020351,0.837711,0.638295,0.815326,0.995633,0.334880,0.178700,0.000406,...,701.603967,703.849507,699.122215,687.917109,711.293560,716.390717,714.357210,827.466633,833.453529,821.740933


## Simple t-test

In [64]:
average_au_intensity_per_video.sessions = average_au_intensity_per_video.index.map(input_class_map)
t, p = average_au_intensity_per_video.iloc[:10].aus().ttest()
pd.DataFrame({"t": t, "p": p}, index= average_au_intensity_per_video.au_columns)

,t,p
mean_AU01,6.630503,9.585643e-05
mean_AU02,32.900581,1.089750e-10
mean_AU04,7.411417,4.054087e-05
mean_AU06,50.006273,2.566064e-12
mean_AU07,27.094823,6.155222e-10
mean_AU10,55.170538,1.062332e-12
mean_AU12,712.489793,1.076048e-22
mean_AU14,16.957212,3.871804e-08
mean_AU15,6.923634,6.883327e-05
mean_AU17,8.490694,1.371510e-05


## Prediction

In [32]:
fex.sessions = fex.input().map(input_class_map)

from sklearn.linear_model import LogisticRegression
clf = fex.predict(X=fex.emotions(), y = fex.sessions, model = LogisticRegression, solver="liblinear")
print("score: ", clf.score(fex.emotions(), fex.sessions))
print("coefficients: ")
display(pd.DataFrame(clf.coef_, columns = fex.emotions().columns))

score:  0.9334741288278775
coefficients: 


,anger,disgust,fear,happiness,sadness,surprise,neutral
0,0.722285,0.566583,2.868222,-8.24894,2.434085,0.096825,3.030539


In [33]:
from sklearn.linear_model import LogisticRegression
clf = fex.predict(X=fex.aus(), y = fex.sessions, model = LogisticRegression, solver="liblinear")
print("score: ", clf.score(fex.aus(), fex.sessions))
print("coefficients: ")
display(pd.DataFrame(clf.coef_, columns = fex.aus().columns))

score:  0.8331573389651531
coefficients: 


,AU01,AU02,AU04,AU06,AU07,AU10,AU12,AU14,AU15,AU17,AU23,AU24
0,-0.042668,-0.669198,-1.326025,-1.102651,-1.671964,-4.342783,2.817245,1.133602,-0.801147,0.004,-0.719024,0.000995


# Regression

In [38]:
fex.sessions = fex.input().map(input_class_map).replace({"gn":.5, "ists":-.5})
X = pd.DataFrame(fex.sessions)
X['intercept'] = 1
b, t, p, df, residuals = fex.regress(X = X, y = fex.emotions())
print("Betas predicting good news estimated for each emotion.")
display(b.round(3).loc[[0]])

Betas predicting good news


/home/jcheong/anaconda3/lib/python3.8/site-packages/nltools/stats.py:970: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]


,anger,disgust,fear,happiness,sadness,surprise,neutral
0,-0.018,-0.015,-0.088,0.619,-0.088,-0.34,-0.07


In [40]:
fex.sessions = fex.input().map(input_class_map).replace({"gn":.5, "ists":-.5})
X = pd.DataFrame(fex.sessions)
X['intercept'] = 1
b, t, p, df, residuals = fex.regress(X = X, y = fex.aus())
print("Betas predicting good news estimated for each AU.")
display(b.round(3).loc[[0]])

Betas predicting good news estimated for each AU.


/home/jcheong/anaconda3/lib/python3.8/site-packages/nltools/stats.py:970: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  stderr = np.sqrt(np.diag(np.linalg.pinv(np.dot(X.T, X))))[:, np.newaxis] * sigma[np.newaxis, :]


,AU01,AU02,AU04,AU06,AU07,AU10,AU12,AU14,AU15,AU17,AU23,AU24
0,0.074,0.111,0.066,0.222,0.246,0.34,-0.004,-0.095,-0.011,-0.0,0.011,0.0
